In [1]:
import numpy as np
import pandas as pd
from Tools import commonfxns as cf, OAPBuoyData as OAP, mplRC, OAPBuoyComp as bc,viz, evalfxns as ev, \
    frequencyBands as fb, AlkFits as AF
pd.set_option('display.max_columns', None)

### create table
- mooring name (short title)
- lat and lon
- number of each type of data points
- whether inclusion criteria met
- whether obs trend was estimated
- *alkalinity fit reference if any*

In [2]:
# load moorings table
dfTable=pd.read_csv('table.csv',index_col=0)

In [3]:
np.unique(dfTable.AlkFit.values)

array(['CullisonGray', 'Fass2016a', 'Fass2016b', 'Fass2017', 'Hunt',
       'Lee', 'None', 'Xue'], dtype=object)

In [4]:
dfTable.keys()

Index(['datasetID', 'shortTitle', 'Lat', 'Lon', 'tos_N', 'tos_N2', 'tos_ofit',
       'tos_mfit', 'sos_N', 'sos_N2', 'sos_ofit', 'sos_mfit', 'spco2_N',
       'spco2_N2', 'spco2_ofit', 'spco2_mfit', 'phos_N', 'phos_N2',
       'phos_ofit', 'phos_mfit', 'chlos_N', 'chlos_N2', 'chlos_ofit',
       'chlos_mfit', 'o2os_N', 'o2os_N2', 'o2os_ofit', 'o2os_mfit', 'phosC_N',
       'phosC_N2', 'phosC_ofit', 'phosC_mfit', 'AlkFit'],
      dtype='object')

In [5]:
citDict={'CullisonGray':r"\citet{CullisonGray2011}", 
         'Fass2016a':r"\citet{Fassbender2016a}", 
         'Fass2016b':r"\citet{Fassbender2016b}", 
         'Fass2017':r"\citet{Fassbender2017}", 
         'Hunt':'C. Hunt, University of New Hampshire, pers. comm. to A. Sutton',
         'Lee':r"\citet{Lee2006}", 
         'Xue':r"\citet{Xue2016}",
         'None':'None'}

In [6]:
dfTable.drop(dfTable.loc[dfTable.shortTitle.isin(['M2','JKEO'])].index,inplace=True)

In [7]:
dfTable['Fit Citation']= [citDict[el] for el in dfTable.AlkFit.values]

In [8]:
dfTable['Region']=[OAP.regions[st] for st in dfTable.shortTitle.values]

In [9]:
dfout=dfTable.loc[dfTable['Fit Citation']!='None',['shortTitle','Region','Fit Citation']].copy(deep=True)
dfout

,shortTitle,Region,Fit Citation
0,Iceland,North Atlantic Ocean,\citet{Lee2006}
1,Gulf of Maine,US east coast,"C. Hunt, University of New Hampshire, pers. co..."
5,BTM,North Atlantic Ocean,\citet{Lee2006}
6,Gray's Reef,US east coast,\citet{Xue2016}
7,Coastal MS,Gulf of Mexico coast,\citet{Lee2006}
8,Coastal LA,Gulf of Mexico coast,\citet{Lee2006}
10,Stratus,Southeast Pacific Ocean,\citet{Lee2006}
12,BOBOA,Indian Ocean,\citet{Lee2006}
13,CCE1,Northeast Pacific Ocean,\citet{CullisonGray2011}
14,CCE2,US west coast,\citet{CullisonGray2011}


In [10]:
trdict={'North Atlantic Ocean':'North Atlantic',
        'Southeast Pacific Ocean':'Southeast Pacific',
        'Northeast Pacific Ocean':'Northeast Pacific',
        'Northwest Pacific Ocean':'Northwest Pacific',
        'Central Pacific Ocean': 'Central Pacific',
        'Equatorial Pacific Ocean':'Equatorial Pacific',
        'Pacific island coral reef': 'Pacific Island CR',
        'Caribbean coral reef': 'Caribbean CR',
        'Atlantic coral reef': 'Atlantic CR',
        'Gulf of Mexico coast': 'Gulf Mex. coast',
        'MOSEAN/WHOTS': 'MOSEAN/ WHOTS'}

In [11]:
def fmt2(ss):
    if type(ss)==str:
        if ss in trdict.keys():
            return trdict[ss]
        else:
            return ss
    elif type(ss)==float:
        return f'{ss:.1f}'
    else:
        return str(ss)

In [12]:
for ind, row in dfout.iterrows():
    print(' & '.join(row.values))

Iceland & North Atlantic Ocean & \citet{Lee2006}
Gulf of Maine & US east coast & C. Hunt, University of New Hampshire, pers. comm. to A. Sutton
BTM & North Atlantic Ocean & \citet{Lee2006}
Gray's Reef & US east coast & \citet{Xue2016}
Coastal MS & Gulf of Mexico coast & \citet{Lee2006}
Coastal LA & Gulf of Mexico coast & \citet{Lee2006}
Stratus & Southeast Pacific Ocean & \citet{Lee2006}
BOBOA & Indian Ocean & \citet{Lee2006}
CCE1 & Northeast Pacific Ocean & \citet{CullisonGray2011}
CCE2 & US west coast & \citet{CullisonGray2011}
Cape Arago & US west coast & \citet{CullisonGray2011}
NH-10 & US west coast & \citet{CullisonGray2011}
Cape Elizabeth & US west coast & \citet{Fassbender2016a}
Cha ba & US west coast & \citet{Fassbender2016a}
Kodiak & Alaskan coast & \citet{Lee2006}
Papa & Northeast Pacific Ocean & \citet{Fassbender2016b}
SOFS & Southern Ocean & \citet{Lee2006}
KEO & Northwest Pacific Ocean & \citet{Fassbender2017}
MOSEAN/WHOTS & Central Pacific Ocean & \citet{Lee2006}
TAO170W

In [13]:
lines=list()
#lines.append(r'   \small'+'\n')
#lines.append(r'   \centering'+'\n')
lines.append(r"""\setlength\tabcolsep{4pt} % default value: 6pt
    \begin{longtable}{llp{0.5\textwidth}}
    \caption{\tblAlkFitsCaption}
    \label{tbl:AlkFits}\\
    \hline 
    """)

head1=' & '.join([r"\multicolumn{1}{l}{"+el+r"}" for el in ['Mooring','Region','Fit Citation']])
lines.append(f'   {head1}'+r' \\'+'\n')
lines.append(r'   \hline'+'\n')
lines.append(r"\endfirsthead"+'\n')

lines.append(r"""\multicolumn{3}{c}%
    {\tablename\ \thetable\ -- \textit{Continued from previous page}} \\
    \hline
    """)
lines.append(f'   {head1}'+r' \\'+'\n')
lines.append(r'   \hline'+'\n')
lines.append(r"\endhead"+'\n')

lines.append(r"""    \hline \multicolumn{3}{r}{\textit{Continued on next page}} \\
    \endfoot
    \hline
    \endlastfoot
    """)

for ind, row in dfout.iterrows():
    r=' & '.join(row.values)
    lines.append(f'   {r} \\\\'+'\n')
lines.append(r'  \end{longtable}')
f = open('TableAlkFits.tex', 'w')
f.writelines(lines)
f.close()